In [1]:
import os
import openai
import pandas as pd
import tiktoken
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI
from langchain.schema import Document
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import Pinecone

In [2]:
# Fine tuning increases the accuracy from 84% to 84.20%; minor change

In [3]:
df = pd.read_csv('../data/csv/processed_toxic_classification_dataset_short.csv')
df['Toxic'] = df['Toxic'].apply(lambda x: str(int(x)))
df

FileNotFoundError: [Errno 2] No such file or directory: '../data/csv/processed_toxic_classification_dataset_short.csv'

In [5]:
load_dotenv()
FINE_TUNED_TOXIC_DETECTION_API_KEY = os.getenv('FINE_TUNED_TOXIC_DETECTION_API_KEY')
pinecone_api_key = os.getenv('PINECONE_API_KEY')
pc_index = os.getenv('PINECONE_GPT')

In [6]:
fine_tuned = OpenAI(
    api_key=FINE_TUNED_TOXIC_DETECTION_API_KEY
)

In [7]:
# initializing Pinecone vector database instance
docSearch = Pinecone(
    index_name=pc_index,
    embedding=OpenAIEmbeddings()
)

C:\Users\dodsone\AppData\Local\Temp\ipykernel_16672\130847528.py:2: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.3 and will be removed in 0.3.0. Use :class:`~PineconeVectorStore` instead.
  docSearch = Pinecone(


In [8]:
def shot_additions(examples):
   evaluation_prompt = ''
   # Read the evaluation prompt from the text file with utf-8 encoding
   with open("../data/text/gpt-3-5-0125-two-shot-prompt.txt", "r", encoding="utf-8") as file:
        evaluation_prompt += file.read() 
   evaluation_prompt += '\n\n'
   for example in examples:
       evaluation_prompt += example

   return evaluation_prompt

In [9]:
def rag_prompt(text):
    two_shots = []
    search_results = docSearch.max_marginal_relevance_search(text, k=25, fetch_k=50)
    for i in search_results:
        content = i.page_content
        if i != 0 and len(content) < 50:
            two_shots.append(content)
    return shot_additions(two_shots)

In [10]:
tuned = pd.DataFrame(columns=['Text', 'Toxic'])

In [11]:
evaluation_prompt = ''
# Read the evaluation prompt from the text file with utf-8 encoding
with open("../data/text/fine_tuned_prompts/gpt-3-5-1106-two-shot-prompt.txt", "r", encoding="utf-8") as file:
    evaluation_prompt += file.read()

In [12]:
evaluation_prompt

'Classify these comments as either toxic (1) non-toxic (0). Toxic comments contain offensive, harmful, or abusive language. Label 1 for toxic and 0 for non-toxic\n\nExamples for Reference:\n\nText: “I disagree with you, but I think you make some valid points.”\n\nLabel: 0\n\nText: “Go back to where you came from, we don’t want you here.”\n\nLabel: 1\n\nClassify the following comment:'

In [13]:
for index, row in df.iterrows():
    text = row['Text']
    completion = fine_tuned.chat.completions.create(
        model="ft:gpt-3.5-turbo-1106:personal::AN7TvIij",
        messages=[
            {
                "role": "system", 
                "content": evaluation_prompt
            },
            {
                "role": "user",
                "content": text
            }
        ]
    )
    tuned.loc[index] = [row['Text'], completion.choices[0].message.content]


In [14]:
tuned

,Text,Toxic
0,Geez forgetful Weve already discussed Marx ana...,0
1,Carioca RFA Thanks support request adminship f...,0
2,Birthday worries Enjoy ur daytalke,0
3,Pseudoscience category Im assuming article pse...,0
4,phrase exists would provided search engine eve...,0
...,...,...
995,2006 UTC rest us ever gone past 3RR good one V...,0
996,Yay lets Pedantic Semantics dance rolls eyes t...,0
997,supposed Know,0
998,guys really discuss napoleon need get fuckin l...,1


In [15]:
compare = tuned['Toxic'] == df['Toxic']
gptthreefive_original_accuracy = compare.values.sum() / compare.size

In [16]:
compare

0       True
1       True
2       True
3       True
4       True
       ...  
995     True
996     True
997     True
998     True
999    False
Name: Toxic, Length: 1000, dtype: bool

In [17]:
# GPT-4o: 97.30% with too mid-detailed evaluation prompt
# GPT-3.5 turbo 50 set: 91.80% with summarized two-shot prompt costing 73 cents
# GPT-3.5 turbo 200 set: 92.00% with summarized two-shot prompt costing 72 cents
# GPT 3.5 turbo 2000 set: 97:30% with summarized two-shot prompt costing 72 cents
print(f"Accuracy: {gptthreefive_original_accuracy * 100:.2f}%")

Accuracy: 97.20%


In [18]:

# completion = fine_tuned.chat.completions.create(
#     model="ft:gpt-4o-2024-08-06:personal::AENT6Q3Y",
#     messages=[
#         {
#             "role": "system", 
#             "content": "The user has a toxicity threshold of 1, meaning that they find 90 to 100 percent of comments non-toxic and have extremely high tolerance. Label this text as either 0 for non-toxic or 1 for toxic."
#         },
#         {
#             "role": "user",
#             "content": df['Text'][998]
#         }
#     ]
# )

In [19]:
completion.choices[0].message.content

'1'